### Sarima model monthly all meters assessed individually with DDH and then summed

### https://towardsdatascience.com/how-to-forecast-sales-with-python-using-sarima-model-ba600992fa7d

In [1]:
import warnings
import itertools
import numpy as np
from numpy import concatenate, savetxt, unique, array, subtract
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
import pandas as pd
from pandas import merge, DataFrame, Series
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import matplotlib
from math import sqrt
from statistics import mean


In [2]:
df = pd.read_csv('CHL_Monthly_Stacked.csv', header=0, sep='[,]', parse_dates=True, squeeze=True, dayfirst=True, engine='python')
meter_ids = unique(df['MeterNo']) 
df1a = pd.DataFrame(index =['Month'], columns=['prediction', 'actual'])
pred_date = '2019-03-01'
print(meter_ids)

[101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136
 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154
 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172
 173 174 175 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251
 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269
 270 271 272 273 274 275 301 302 303 304 305 306 307 308 309 310 311 312
 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330
 331 332 333 334 335 336 337 338 339 340 341 342 343 344 345 346 347 348
 349 350 351 352 353 354 355 356 357 358 359 360 361 362 363 364 365 366
 367 368 369 370 371 372 373 374 375 401 402 403 404 405 406 407 408 409
 410 411 412 413 414 415 416 417 418 419 420 421 42

In [3]:
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print('Examples of parameter for SARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

Examples of parameter for SARIMA...
SARIMAX: (0, 0, 1) x (0, 0, 1, 12)
SARIMAX: (0, 0, 1) x (0, 1, 0, 12)
SARIMAX: (0, 1, 0) x (0, 1, 1, 12)
SARIMAX: (0, 1, 0) x (1, 0, 0, 12)


In [4]:
for m in meter_ids:
    df_meters = pd.read_csv('CHL_Monthly_Stacked.csv', header=0, sep='[,]', parse_dates=True, 
                            squeeze=True, dayfirst=True, engine='python')
    df_meters['Date'] = pd.to_datetime(df_meters['Date'], dayfirst=True)
    df_meters.columns =['MeterNo', 'Month', 'kWh']
    y = df_meters.set_index('Month')
    df_DDH = pd.read_csv('DDH_Monthly.csv', header=0, sep='[,]', parse_dates=True, squeeze=True, dayfirst=True, engine='python')
    df_DDH['Date'] = pd.to_datetime(df_DDH['Date'], dayfirst=True)
    df_DDH.columns =['Month', 'DDH']
    z = df_DDH.set_index('Month')
    y = merge(y, z, left_index=True, right_index=True)
    y = y.loc[y['MeterNo'] == m]
    y.drop(y.columns[[0]], axis = 1, inplace = True)
    endog = y.iloc[:,0]
    exog = y.iloc[:,1:]
    freq = 'MS' # month start
    mod = sm.tsa.statespace.SARIMAX(endog = endog,
                                order=(1, 1, 1),
                                seasonal_order=(0, 1, 1, 12),
                                exog = exog,
                                freq = freq,
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results = mod.fit()
    pred = results.get_prediction(start=pd.to_datetime(pred_date), dynamic=False)
    y_forecasted = pred.predicted_mean
    y_forecasted =y_forecasted.to_frame()
    y_truth = y[pred_date:]
    df1 = merge(y_forecasted, y_truth, left_index=True, right_index=True)
    df1.drop(df1.columns[[2]], axis = 1, inplace = True)
    df1.columns =['prediction', 'actual']
    df1['error'] = round(((df1['prediction'] - df1['actual'])),2)
    df1a = df1a.append(df1)


C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information 

C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information 

C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information 

C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency informat

C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information 

C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information 

C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information 

C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information 

In [5]:
# Each meter per month
df2 = df1a.dropna()
print(df2)

             prediction actual   error
2019-03-01   107.877191    137  -29.12
2019-04-01   128.817369    134   -5.18
2019-05-01   112.901441     92   20.90
2019-06-01    62.190979     57    5.19
2019-07-01    53.259832     53    0.26
...                 ...    ...     ...
2020-10-01   443.632766    653 -209.37
2020-11-01  1039.573443   1114  -74.43
2020-12-01  1446.714352   1721 -274.29
2021-01-01  2001.352621   1797  204.35
2021-02-01  1717.626132   1719   -1.37

[6840 rows x 3 columns]


In [6]:
mse = mean_squared_error(df2['actual'],df2['prediction'])
print(mse)

5774.442343435007


In [7]:
rmse = np.sqrt(mse)
print(rmse)

75.9897515684517


In [8]:
# Summary of each meter for each month
def Summary(x):
    return round(Series(index=['min','max', 'mean','sum','count'],data=[x.min(),x.max(),x.mean(),x.sum(),x.count()]),2)
df2.apply(Summary)

,prediction,actual,error
min,-1978.65,-1821.00,-1256.65
max,2226.51,2152.00,1252.51
mean,365.69,367.82,-2.13
sum,2501294.55,2515858.00,-14563.48
count,6840.00,6840.00,6840.00


In [9]:
#Mean Square Error Calc
def mse(g):
    mse = mean_squared_error(g['actual'], g['prediction'])
    return Series({'mse':mse})

df2a = df2.groupby(df2.index)['prediction'].sum()
df2b = df2.groupby(df2.index)['actual'].sum()
df2c = df2.groupby(df2.index).apply(mse).reset_index()
df2c = df2c.set_index('index')
df3 = merge(df2a, df2b, left_index=True, right_index=True)
df3 = merge(df3, df2c, left_index=True, right_index=True)
df3['rmse'] = round(np.sqrt(df3['mse']),2)
df3['error'] = round(((df3['prediction'] - df3['actual'])),2)
df3['%error'] = (df3['error']/df3['actual'])*100
df3.columns =['sum of predictions', 'sum of actuals', 'mse', 'rmse', 'error', '%error']
print(df3)

            sum of predictions  sum of actuals           mse    rmse  \
2019-03-01       109964.309090          123163  13694.223983  117.02   
2019-04-01       107157.954553          109928   4174.694263   64.61   
2019-05-01        87670.841233           86331   5580.933009   74.71   
2019-06-01        68797.835476           64729   1822.531490   42.69   
2019-07-01        57404.118087           60202   2347.578453   48.45   
2019-08-01        58122.115334           55487    881.265075   29.69   
2019-09-01        66693.543850           73882   4549.934310   67.45   
2019-10-01        92138.797970           91639   1279.492137   35.77   
2019-11-01       122724.268896          119355   5807.643949   76.21   
2019-12-01       146798.347869          152017   2124.419147   46.09   
2020-01-01       158678.284342          157644  10413.661412  102.05   
2020-02-01       148084.523719          153167   2745.101651   52.39   
2020-03-01       144315.421574          132075  10932.485817  10

In [10]:
df3.apply(Summary)

,sum of predictions,sum of actuals,mse,rmse,error,%error
min,57404.12,55487.00,881.27,29.69,-16017.43,-16.40
max,171006.79,159650.00,17519.58,132.36,12240.42,9.84
mean,104220.61,104827.42,5774.44,69.98,-606.81,-0.49
sum,2501294.55,2515858.00,138586.62,1679.54,-14563.45,-11.80
count,24.00,24.00,24.00,24.00,24.00,24.00


for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y,order=param,seasonal_order=param_seasonal,enforce_stationarity=False,enforce_invertibility=False, period=1)
            results = mod.fit()
            print('ARIMA{}x{}12 - AIC:{}'.format(param,param_seasonal,results.aic))
        except: 
            continue

for m in meter_ids:
    df = pd.read_csv('CHL_Monthly_Stacked.csv', header=0, sep='[,]', parse_dates=True, squeeze=True, dayfirst=True, engine='python')
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
    df.columns =['MeterNo', 'WeekNo', 'kWh']
    y = df.set_index(['WeekNo'])
    y = y.loc[y['MeterNo'] == m]
    y.drop(y.columns[[0]], axis = 1, inplace = True)
    #print(y)
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                mod = sm.tsa.statespace.SARIMAX(y,order=param,seasonal_order=param_seasonal,enforce_stationarity=False,enforce_invertibility=False)
                results = mod.fit()
                print('ARIMA{}x{}12 - AIC:{}'.format(param,param_seasonal,results.aic))
            except: 
                continue